In [3]:
import requests
import pandas as pd
from io import StringIO
import yfinance as yf
import time

In [4]:
def getUpcomingTickers():
    apikey = "EOZ1S7LT4L6X8IVE"

    url = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=' + apikey
    response = requests.get(url)

    if response.status_code != 200:
        print("Error: unable to retrieve data")
        exit()
    print(response.content)

    raw_data = StringIO(response.text)
    df = pd.read_csv(raw_data)
    filtered_df = df[df['currency'] == 'USD'][["symbol","name","reportDate"]]
    filtered_df["reportDate"] = pd.to_datetime(filtered_df["reportDate"])
    filtered_df = filtered_df.sort_values(by=["reportDate", "name"])

    return filtered_df

In [ ]:
#### TESTING
apikey = "EOZ1S7LT4L6X8IVE"

url = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=' + apikey
response = requests.get(url)

if response.status_code != 200:
    print("Error: unable to retrieve data")
    exit()
print(response.content)

raw_data = StringIO(response.text)
df = pd.read_csv(raw_data)

In [ ]:
filtered_df = df[df['currency'] == 'USD'][["symbol","name","reportDate","currency"]]
filtered_df["reportDate"] = pd.to_datetime(filtered_df["reportDate"])


date_sliced_df = filtered_df[filtered_df['reportDate'] <= '2024-11-10']
print(date_sliced_df.shape)

In [7]:
def getHistoricalERsForTicker(ticker,index):
    if index % 10 == 0:
        print(f"Processing {index + 1}/{len(date_sliced_df)}: FETCHING {ticker}")
    try:
        stock = yf.Ticker(ticker)
        earnings_calendar = stock.earnings_dates
        print(earnings_calendar)
        earnings_calendar.index = pd.to_datetime(earnings_calendar.index).tz_convert('America/New_York')
        current_time = pd.Timestamp.now(tz='America/New_York')
        past_earnings = earnings_calendar[earnings_calendar.index < current_time]
        last_12_reports = past_earnings.sort_index(ascending=False).head(12).index.tolist()
        percentChanges = []
        market_close_time = pd.Timestamp("16:00:00").time()
        for timestamp in last_12_reports:
            ER_time = timestamp.time()
            ER_date = timestamp.date()
            if ER_time >= market_close_time:
                #aftermarket report
                closePrice1 = yf.download(ticker, start=ER_date, end=ER_date + pd.Timedelta(days=1),progress=False)['Close']
                closePrice1 = closePrice1.iloc[0] if not closePrice1.empty else None
                
                closePrice2 = yf.download(ticker, start=ER_date + pd.Timedelta(days=1), end=ER_date + pd.Timedelta(days=2),progress=False)['Close']
                closePrice2 = closePrice2.iloc[0] if not closePrice2.empty else None
                
                if closePrice1.item() is not None and closePrice1.item() != 0:
                    percentChange = (closePrice2 - closePrice1) / closePrice1
                    percentChanges.append(f"{round(percentChange.item() * 100, 2)}%")
            else:
                #premarket report
                closePrice1 = yf.download(ticker, start=ER_date - pd.Timedelta(days=1), end=ER_date,progress=False)['Close']
                closePrice1 = closePrice1.iloc[0] if not closePrice1.empty else None
                
                closePrice2 = yf.download(ticker, start=ER_date, end=ER_date + pd.Timedelta(days=1),progress=False)['Close']
                closePrice2 = closePrice2.iloc[0] if not closePrice2.empty else None
                
                if closePrice1.item() is not None and closePrice1.item() != 0:
                    percentChange = (closePrice2 - closePrice1) / closePrice1
                    percentChanges.append(f"{round(percentChange.item() * 100, 2)}%")
    except Exception as e:
        percentChanges = []
    time.sleep(1)
    return percentChanges

In [8]:
print("Fetching upcoming tickers...")
upcoming_tickers_df = getUpcomingTickers()

Fetching upcoming tickers...
b'symbol,name,reportDate,fiscalDateEnding,estimate,currency\r\nA,Agilent Technologies Inc,2024-11-25,2024-10-31,1.41,USD\r\nAA,Alcoa Corp,2025-01-15,2024-12-31,0.8,USD\r\nAAALY,AAALY,2024-11-07,2024-09-30,,EUR\r\nAABVF,AABVF,2024-12-12,2024-10-31,,CAD\r\nAACG,ATA Creativity Global,2024-11-07,2024-09-30,,CNY\r\nAACT,Ares Acquisition Corporation II - Class A,2024-11-05,2024-09-30,,USD\r\nAACTF,AACTF,2024-11-27,2024-09-30,,USD\r\nAADI,Aadi Bioscience Inc,2024-11-06,2024-09-30,-0.46,USD\r\nAAFRF,AAFRF,2025-01-30,2024-12-31,,USD\r\nAAGFF,AAGFF,2024-11-28,2024-08-31,,USD\r\nAAHIF,AAHIF,2024-12-23,2024-11-30,,USD\r\nAAIC,AAIC,2024-11-11,2024-09-30,,USD\r\nAAL,American Airlines Group Inc,2025-01-23,2024-12-31,0.4,USD\r\nAAMC,AAMC,2024-11-12,2024-09-30,,USD\r\nAAMCF,AAMCF,2024-11-12,2024-09-30,,USD\r\nAAME,Atlantic American Corp,2024-11-11,2024-09-30,,USD\r\nAANNF,AANNF,2024-11-27,2024-09-30,,USD\r\nAAOI,Applied Optoelectronics Inc,2024-11-07,2024-09-30,-0.17,USD\r\

In [9]:
date_sliced_df = upcoming_tickers_df[upcoming_tickers_df['reportDate'] <= '2024-11-10']
print(date_sliced_df.shape)

(2509, 3)


In [10]:
date_sliced_df = date_sliced_df.reset_index(drop=True)
date_sliced_df['percentChanges'] = date_sliced_df.apply(lambda row: getHistoricalERsForTicker(row['symbol'], row.name), axis=1)

Processing 1/2509: FETCHING XXII
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-08-11 06:00:00-04:00           NaN           NaN          NaN
2025-05-13 06:00:00-04:00           NaN           NaN          NaN
2025-03-26 06:00:00-04:00           NaN           NaN          NaN
2024-11-04 08:00:00-05:00           NaN           NaN          NaN
2024-08-13 06:00:00-04:00           NaN           NaN          NaN
2024-05-15 06:00:00-04:00         -5.31         -2.94       0.4465
2024-03-28 07:00:00-04:00         -8.88        -18.40      -1.0721
2023-11-06 07:00:00-05:00         -0.92         -3.68      -2.9971
2023-08-14 06:00:00-04:00         -1.06         -1.40      -0.3233
2023-05-09 06:00:00-04:00         -0.07         -0.08      -0.0667
2023-03-09 06:00:00-05:00         -0.06         -0.12      -1.0000
2022-11-08 06:00:00-05:00         -0.06         -0.06          NaN



1 Failed download:
['XXII']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-11-05 -> 2023-11-06)')


                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-08-05 16:00:00-04:00           NaN           NaN          NaN
2025-05-06 06:00:00-04:00           NaN           NaN          NaN
2025-01-29 05:00:00-05:00           NaN           NaN          NaN
2024-11-04 16:00:00-05:00          0.08           NaN          NaN
2024-08-07 16:00:00-04:00          0.09          0.08      -0.1061
2024-05-08 16:00:00-04:00          0.06          0.08       0.2380
2024-01-31 16:00:00-05:00          0.10          0.12       0.1729
2023-11-01 16:00:00-04:00          0.09          0.14       0.6397
2023-08-08 16:00:00-04:00          0.13          0.13       0.0349
2023-05-11 16:00:00-04:00          0.09          0.11       0.1839
2023-02-01 16:00:00-05:00          0.03          0.07       1.6555
2022-10-27 16:00:00-04:00          0.04          0.05       0.1820
                           EPS Estimate  Reported EPS  Surpris

KeyboardInterrupt: 

In [49]:
copyOfDf = date_sliced_df.copy()

In [50]:
copyOfDf = copyOfDf[copyOfDf['percentChanges'].apply(bool)]


In [61]:
def mean_absolute_percent_change(percentages):
    abs_values = [abs(float(p.strip('%'))) for p in percentages]
    return str(round(sum(abs_values) / len(abs_values),2)) + "%"

In [1]:
def getMarketCap(ticker):
    print(ticker)
    try:
        stock = yf.Ticker(ticker)
        return stock.info['marketCap']
    except:
        return None

In [2]:
copyOfDf['meanAbsolutePercentChange'] = copyOfDf['percentChanges'].apply(mean_absolute_percent_change)
copyOfDf['marketCap'] = copyOfDf['symbol'].apply(getMarketCap)

NameError: name 'copyOfDf' is not defined

In [63]:
print(copyOfDf.head(10))

    symbol     name reportDate  \
1     EGHT  8X8 Inc 2024-11-04   
2     ACBM     ACBM 2024-11-04   
3    ACRXF    ACRXF 2024-11-04   
4     ADXS     ADXS 2024-11-04   
6   AGM-PD   AGM-PD 2024-11-04   
7   AGM-PE   AGM-PE 2024-11-04   
8   AGM-PF   AGM-PF 2024-11-04   
9   AGM-PG   AGM-PG 2024-11-04   
10   AGMJF    AGMJF 2024-11-04   
11  AHH-PA   AHH-PA 2024-11-04   

                                       percentChanges  \
1   [-25.97%, 20.09%, -11.61%, 36.24%, -21.62%, -3...   
2                                              [0.0%]   
3                                        [0.0%, 0.0%]   
4   [0.0%, 6.43%, 2.15%, 4.27%, -1.72%, 3.46%, -6....   
6                                             [1.34%]   
7                                             [1.69%]   
8                                             [0.65%]   
9                                             [2.01%]   
10                                           [-3.29%]   
11                                            [1.88%]  

In [64]:
copyOfDf.to_csv("upcomingEarningsReport.csv", index=False)